# Text Mining

## TF-IDF Calculation for the data and Information Gain feature extraction

## Aarthi Mahalakshmi Shankar

In [1]:
import pandas as pd 
import numpy as np
from numpy import shape
import re
import string  
import nltk
from nltk.corpus import stopwords
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer   
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn import metrics
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics  import confusion_matrix

from sklearn.metrics import classification_report
from scipy.sparse import csr_matrix, hstack
from scipy import sparse 
tokenizer = TweetTokenizer()
nltk.download('stopwords')

from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

eng_stopwords = set(stopwords.words("english"))

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aarthi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/Users/aarthi/libpff/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df=pd.read_csv("train.csv")

In [3]:
df['comment_text'].fillna("unknown", inplace=True)

In [4]:
#function to clean the comments-dataset
def clean(comment):
    #Converting to lower case 
    comment=comment.lower()
    #removing '\n'
    comment=re.sub("\\n","",comment)
    # removing ip,user
    comment=re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}","",comment)
    #removing usernames
    comment=re.sub("\[\[.*\]","",comment)
    
    word_list=tokenizer.tokenize(comment)
    word_list = [w for w in word_list if not w in eng_stopwords]
    
    clean_sent=" ".join(word_list)
    #removing non alphanum,digit character
    clean_sent=re.sub("\W+"," ",clean_sent)
    clean_sent=re.sub("  "," ",clean_sent)
    return(clean_sent)

In [5]:
df

,id,comment_text,toxic,obscene
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0


In [6]:
df['comment_text'] = df['comment_text'].apply(lambda x :clean(x))

In [7]:
df

,id,comment_text,toxic,obscene
0,0000997932d777bf,explanationwhy edits made username hardcore me...,0,0
1,000103f0d9cfb60f,d aww matches background colour i m seemingly ...,0,0
2,000113f07ec002fd,hey man i m really trying edit war guy constan...,0,0
3,0001b41b1c6bb37e,morei can t make real suggestions improvement...,0,0
4,0001d958c54c6e35,sir hero chance remember page that s,0,0
5,00025465d4725e87,congratulations well use tools well talk,0,0
6,0002bcb3da6cb337,cocksucker piss around work,1,1
7,00031b1e95af7921,vandalism matt shirvington article reverted pl...,0,0
8,00037261f536c51d,sorry word nonsense offensive anyway i m inten...,0,0
9,00040093b2687caa,alignment subject contrary dulithgow,0,0


In [8]:
df['words'] = df['comment_text'].str.strip().str.split('[\W_]+')

In [9]:
df

,id,comment_text,toxic,obscene,words
0,0000997932d777bf,explanationwhy edits made username hardcore me...,0,0,"[explanationwhy, edits, made, username, hardco..."
1,000103f0d9cfb60f,d aww matches background colour i m seemingly ...,0,0,"[d, aww, matches, background, colour, i, m, se..."
2,000113f07ec002fd,hey man i m really trying edit war guy constan...,0,0,"[hey, man, i, m, really, trying, edit, war, gu..."
3,0001b41b1c6bb37e,morei can t make real suggestions improvement...,0,0,"[morei, can, t, make, real, suggestions, impro..."
4,0001d958c54c6e35,sir hero chance remember page that s,0,0,"[sir, hero, chance, remember, page, that, s]"
5,00025465d4725e87,congratulations well use tools well talk,0,0,"[congratulations, well, use, tools, well, talk]"
6,0002bcb3da6cb337,cocksucker piss around work,1,1,"[cocksucker, piss, around, work]"
7,00031b1e95af7921,vandalism matt shirvington article reverted pl...,0,0,"[vandalism, matt, shirvington, article, revert..."
8,00037261f536c51d,sorry word nonsense offensive anyway i m inten...,0,0,"[sorry, word, nonsense, offensive, anyway, i, ..."
9,00040093b2687caa,alignment subject contrary dulithgow,0,0,"[alignment, subject, contrary, dulithgow]"


In [10]:
rows = list()
for row in df[['id', 'words','toxic','obscene']].iterrows():
    r = row[1]
    for word in r.words:
        rows.append((r.id, word,r.toxic,r.obscene))

words = pd.DataFrame(rows, columns=['id', 'word','toxic','obscene'])
words.head()

,id,word,toxic,obscene
0,0000997932d777bf,explanationwhy,0,0
1,0000997932d777bf,edits,0,0
2,0000997932d777bf,made,0,0
3,0000997932d777bf,username,0,0
4,0000997932d777bf,hardcore,0,0


In [11]:
words = words[words.word.str.len() > 0]
words.head()

,id,word,toxic,obscene
0,0000997932d777bf,explanationwhy,0,0
1,0000997932d777bf,edits,0,0
2,0000997932d777bf,made,0,0
3,0000997932d777bf,username,0,0
4,0000997932d777bf,hardcore,0,0


In [12]:
words['word'] = words.word.str.lower()
words.head()

,id,word,toxic,obscene
0,0000997932d777bf,explanationwhy,0,0
1,0000997932d777bf,edits,0,0
2,0000997932d777bf,made,0,0
3,0000997932d777bf,username,0,0
4,0000997932d777bf,hardcore,0,0


In [13]:
words_tfidf = words.copy(deep = True)
words_tfidf.head()

,id,word,toxic,obscene
0,0000997932d777bf,explanationwhy,0,0
1,0000997932d777bf,edits,0,0
2,0000997932d777bf,made,0,0
3,0000997932d777bf,username,0,0
4,0000997932d777bf,hardcore,0,0


In [14]:
counts = words_tfidf.groupby('id')\
    .word.value_counts()\
    .to_frame()\
    .rename(columns={'word':'n_w'})
counts.head()

n_w
id               word               
0000997932d777bf closure           1
                 dolls             1
                 edits             1
                 explanationwhy    1
                 fac               1

In [15]:
word_sum = counts.groupby(level=0)\
    .sum()\
    .rename(columns={'n_w': 'n_d'})
word_sum

,n_d
id,
0000997932d777bf,25
000103f0d9cfb60f,17
000113f07ec002fd,23
00013fa6fb6ef643,33
0001b41b1c6bb37e,57
0001d958c54c6e35,7
00024b59235015f3,22
00025465d4725e87,6
0002bcb3da6cb337,4


In [16]:
tf = counts.join(word_sum)

tf['tf'] = tf.n_w/tf.n_d

tf.head()

n_w  n_d    tf
id               word                          
0000997932d777bf closure           1   25  0.04
                 dolls             1   25  0.04
                 edits             1   25  0.04
                 explanationwhy    1   25  0.04
                 fac               1   25  0.04

In [17]:
c_d = words.word.nunique()
c_d

204606

In [18]:
idf = words_tfidf.groupby('word')\
    .id\
    .nunique()\
    .to_frame()\
    .rename(columns={'id':'i_d'})\
    .sort_values('i_d')
idf.head()

,i_d
word,
kking,1
hyphenned,1
hyphenitis,1
wjw,1
russellis,1


In [19]:
idf['idf'] = np.log(c_d/idf.i_d.values)

idf.head()

,i_d,idf
word,,
kking,1,12.228841
hyphenned,1,12.228841
hyphenitis,1,12.228841
wjw,1,12.228841
russellis,1,12.228841


In [20]:
tf_idf = tf.join(idf)

tf_idf.head()

n_w  n_d    tf   i_d        idf
id               word                                           
0000997932d777bf closure           1   25  0.04    79   7.859394
                 dolls             1   25  0.04    19   9.284402
                 edits             1   25  0.04  7400   3.319606
                 explanationwhy    1   25  0.04     1  12.228841
                 fac               1   25  0.04   255   6.687578

In [21]:
tf_idf['tf_idf'] = tf_idf.tf * tf_idf.idf
tf_idf.head()

n_w  n_d    tf   i_d        idf    tf_idf
id               word                                                     
0000997932d777bf closure           1   25  0.04    79   7.859394  0.314376
                 dolls             1   25  0.04    19   9.284402  0.371376
                 edits             1   25  0.04  7400   3.319606  0.132784
                 explanationwhy    1   25  0.04     1  12.228841  0.489154
                 fac               1   25  0.04   255   6.687578  0.267503

In [22]:
tf_idf_copy = tf_idf.copy(deep=True)

In [23]:
tf_idf_copy = tf_idf_copy.reset_index(drop=False)

In [24]:
tf_idf_copy

,id,word,n_w,n_d,tf,i_d,idf,tf_idf
0,0000997932d777bf,closure,1,25,0.040000,79,7.859394,0.314376
1,0000997932d777bf,dolls,1,25,0.040000,19,9.284402,0.371376
2,0000997932d777bf,edits,1,25,0.040000,7400,3.319606,0.132784
3,0000997932d777bf,explanationwhy,1,25,0.040000,1,12.228841,0.489154
4,0000997932d777bf,fac,1,25,0.040000,255,6.687578,0.267503
5,0000997932d777bf,fan,1,25,0.040000,741,5.620841,0.224834
6,0000997932d777bf,gas,1,25,0.040000,211,6.876983,0.275079
7,0000997932d777bf,hardcore,1,25,0.040000,97,7.654130,0.306165
8,0000997932d777bf,i,1,25,0.040000,24002,2.142949,0.085718
9,0000997932d777bf,m,1,25,0.040000,13852,2.692657,0.107706


In [25]:
shape(tf_idf_copy)

(4634093, 8)

In [26]:
shape(tf_idf_copy.tf_idf)

(4634093,)

In [27]:
shape(words_tfidf.iloc[:,2:4])

(5840030, 2)

In [28]:
tf_idf_copy

,id,word,n_w,n_d,tf,i_d,idf,tf_idf
0,0000997932d777bf,closure,1,25,0.040000,79,7.859394,0.314376
1,0000997932d777bf,dolls,1,25,0.040000,19,9.284402,0.371376
2,0000997932d777bf,edits,1,25,0.040000,7400,3.319606,0.132784
3,0000997932d777bf,explanationwhy,1,25,0.040000,1,12.228841,0.489154
4,0000997932d777bf,fac,1,25,0.040000,255,6.687578,0.267503
5,0000997932d777bf,fan,1,25,0.040000,741,5.620841,0.224834
6,0000997932d777bf,gas,1,25,0.040000,211,6.876983,0.275079
7,0000997932d777bf,hardcore,1,25,0.040000,97,7.654130,0.306165
8,0000997932d777bf,i,1,25,0.040000,24002,2.142949,0.085718
9,0000997932d777bf,m,1,25,0.040000,13852,2.692657,0.107706


In [29]:
X, Y = words.word , words.iloc[:,2:4]
cv = CountVectorizer(max_df=0.95, min_df=2,
                                     max_features=10000,
                                     stop_words='english')

In [30]:
X_vec = cv.fit_transform(X)

In [31]:
X_vec

<5840030x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 4297630 stored elements in Compressed Sparse Row format>

In [32]:
# Function to perform training with giniIndex.
def train_using_gini(X_train, X_test, train_y):
 
    # Creating the classifier object
    clf_gini = DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=3, min_samples_leaf=5)
 
    clf_gini.fit(X_train, train_y)
    return clf_gini

In [33]:
# Function to perform training with entropy.
def train_using_entropy(X_train, X_test, train_y):
 
    # Decision tree with entropy
    clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100,max_depth = 3, min_samples_leaf = 5)
 
    clf_entropy.fit(X_train, train_y)
    return clf_entropy

In [34]:
# Function to make predictions
def prediction(X_test, clf_object):
 
    y_pred = clf_object.predict(X_test)
  
    return y_pred

In [35]:
# Function to calculate accuracy
def cal_accuracy(y_test, y_pred):
     
    print("Confusion Matrix:\n ", confusion_matrix(y_test, y_pred))
     
    print ("Accuracy : \n", accuracy_score(y_test,y_pred)*100)
     
    print("Report : \n", classification_report(y_test, y_pred))

In [36]:
X_train, X_test, y_train, y_test = train_test_split( 
          X_vec, Y, test_size = 0.3, random_state = 100)

In [ ]:
X_test

In [53]:
clf_gini_toxic = train_using_gini(X_train, X_test, y_train['toxic'])
clf_entropy_toxic = train_using_entropy(X_train, X_test, y_train['toxic'])
     
print("Results Using Gini Index: TOXIC ")
     
    # Prediction using gini
y_train_pred_gini = prediction(X_train, clf_gini_toxic)
y_test_pred_gini = prediction(X_test, clf_gini_toxic)
cal_accuracy(y_train['toxic'], y_train_pred_gini)
cal_accuracy(y_test['toxic'], y_test_pred_gini)
     
print("Results Using Information Gain: TOXIC ")
    # Prediction using Information Gain
y_train_pred_entropy = prediction(X_train, clf_entropy_toxic)
y_test_pred_entropy = prediction(X_test, clf_entropy_toxic)
cal_accuracy(y_train['toxic'], y_train_pred_entropy)
cal_accuracy(y_test['toxic'], y_test_pred_entropy)


clf_gini_obscene = train_using_gini(X_train, X_test, y_train['obscene'])
clf_entropy_obscene = train_using_entropy(X_train, X_test, y_train['obscene'])
     

print("Results Using Gini Index: OBSCENE ")
     
    # Prediction using gini
y_train_pred_gini = prediction(X_train, clf_gini_obscene)
y_test_pred_gini = prediction(X_test, clf_gini_obscene)
cal_accuracy(y_train['obscene'], y_train_pred_gini)
cal_accuracy(y_test['obscene'], y_test_pred_gini)
     
print("Results Using Information Gain: OBSCENE ")
    # Prediction using Information Gain
y_train_pred_entropy = prediction(X_train, clf_entropy_obscene)
y_test_pred_entropy = prediction(X_test, clf_entropy_obscene)
cal_accuracy(y_train['obscene'], y_train_pred_entropy)
cal_accuracy(y_test['obscene'], y_test_pred_entropy)
     

Results Using Gini Index: TOXIC 
Confusion Matrix:
  [[3774339     366]
 [ 301751   11565]]
Accuracy : 
 92.6097003905
Report : 
              precision    recall  f1-score   support

          0       0.93      1.00      0.96   3774705
          1       0.97      0.04      0.07    313316

avg / total       0.93      0.93      0.89   4088021

Confusion Matrix:
  [[1617255     162]
 [ 129606    4986]]
Accuracy : 
 92.5931887336
Report : 
              precision    recall  f1-score   support

          0       0.93      1.00      0.96   1617417
          1       0.97      0.04      0.07    134592

avg / total       0.93      0.93      0.89   1752009

Results Using Information Gain: TOXIC 
Confusion Matrix:
  [[3774377     328]
 [ 301793   11523]]
Accuracy : 
 92.6096025436
Report : 
              precision    recall  f1-score   support

          0       0.93      1.00      0.96   3774705
          1       0.97      0.04      0.07    313316

avg / total       0.93      0.93      0.89   4